### <b> Importing Libraries </b>

In [180]:
import pickle
from torch_geometric.data import DataLoader
import torch_geometric
import torch.nn as nn
import torch
import random

### <b>Graph Datasets</b>

In [181]:
graphs=pickle.load(open("data/graphs.pkl","rb"))

In [182]:
number_range = 671 # 0 to 671 inclusive
set_size = 67

val_class_0 = set(random.sample(range(number_range), set_size))
test_class_0 = set(random.sample(set(range(number_range)) - set(val_class_0), set_size))

val_class_1 = set(random.sample(range(672,len(graphs)), set_size))
test_class_1 = set(random.sample(set(range(672,len(graphs))) - set(val_class_1), set_size))


result_pipe = val_class_0 | val_class_1 | test_class_0 | test_class_1
train_set=set(range(len(graphs)))-result_pipe

len(val_class_0),len(val_class_1),len(test_class_0),len(test_class_1),len(train_set)

C:\Users\rbw19\AppData\Local\Temp\ipykernel_30184\3907160027.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_class_0 = set(random.sample(set(range(number_range)) - set(val_class_0), set_size))
C:\Users\rbw19\AppData\Local\Temp\ipykernel_30184\3907160027.py:8: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_class_1 = set(random.sample(set(range(672,len(graphs))) - set(val_class_1), set_size))


(67, 67, 67, 67, 1078)

In [207]:
validation=val_class_0.union(val_class_1)
test=test_class_0.union(test_class_1)
set.intersection(validation,test,train_set)

set()

If there is no intersection then there is no intersection between train,test and validatio

In [209]:
val_data=[graphs[i] for i in val_class_0]+[graphs[i] for i in val_class_1]
test_data=[graphs[i] for i in test_class_0]+[graphs[i] for i in test_class_1]
train_data=[graphs[i] for i in train_set]

In [210]:
train_dataset = DataLoader(train_data, batch_size=10, shuffle=True) # splitting with the batch size of 2
val_dataset= DataLoader(val_data, shuffle=True) 
test_dataset=DataLoader(test_data, shuffle=True)

c:\Users\rbw19\OneDrive\Desktop\GNN_guide\graph_venv\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [219]:
import torch

# Assuming you have a tensor like the one you provided
tensor = torch.tensor([[False], [False], [True], [False], [False], [False], [False], [False], [False], [False]])

# Count the number of True values
count_true = tensor.sum().item()

print(f"Number of True values: {count_true}"," \n No of false values :",len(tensor)-count_true ,"\n Accuracy :",count_true/(len(tensor)-count_true))


Number of True values: 1  
 No of false values : 9 
 Accuracy : 0.1111111111111111


### <b> Model creation and trainning</b>

In [237]:
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.data import DataLoader

# Assuming you have 'dataset' and 'val_data' DataLoader objects defined

# Define a simple Graph Neural Network (GNN) model
class GNNModel(nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(15, 8)
        self.conv2 = torch_geometric.nn.GCNConv(8, 3)
        self.conv3 = torch_geometric.nn.GCNConv(3, 1)
        self.fc = nn.Linear(3, 1) 

    def forward(self, batch_data):
        x, edge_index = batch_data.x, batch_data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        # x = self.conv3(x, edge_index)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

model = GNNModel()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch in dataset:
        target = batch.y
        optimizer.zero_grad()
        output = model(batch)

        predictions = torch.round(torch.sigmoid(output))
    
        loss = criterion(predictions.squeeze(), target.float())
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for val_batch in val_data:
            val_output = model(val_batch)
            val_predictions = torch.round(torch.sigmoid(val_output))
            labels = val_batch.y.view(-1, 1)
            tensor =(val_predictions == labels)
            # count_true = tensor.sum().item()
            
            correct += (val_predictions == labels).sum().item() #count how many correct predictions are there
            total += labels.size(0)

    print(correct,total)
    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {val_accuracy}')


940 1309
Epoch 1/50, Loss: 0.6931472420692444, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 2/50, Loss: 0.6931471824645996, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 3/50, Loss: 0.6931471824645996, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 4/50, Loss: 0.6931472420692444, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 5/50, Loss: 0.6931472420692444, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 6/50, Loss: 0.6931471824645996, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 7/50, Loss: 0.6931471228599548, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 8/50, Loss: 0.6931472420692444, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 9/50, Loss: 0.6931472420692444, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 10/50, Loss: 0.6931472420692444, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 11/50, Loss: 0.6931471228599548, Validation Accuracy: 0.7181054239877769
940 1309
Epoch 12/50, Loss: 0.69314718246

### <b> Testing and Accuracy measurements <b>

In [260]:
correct = 0
total = 0
tp = 0  # True Positive
tn = 0  # True Negative
fp = 0  # False Positive
fn = 0  # False Negative

with torch.no_grad():
    for val_batch_batch in test_dataset:
        val_output = model(val_batch)
        val_predictions = torch.round(torch.sigmoid(val_output))
        labels = val_batch.y.view(-1, 1)
        tensor =(val_predictions == labels)
        # count_true = tensor.sum().item()
        # Count true positive, true negative, false positive, and false negative
        # print((val_predictions == 1) & (labels == 1))
        tp += ((val_predictions == True) & (labels == True)).sum().item()
        
        tn += ((val_predictions == False) & (labels == False)).sum().item()
        fp += ((val_predictions == True) & (labels == False)).sum().item()
        fn += ((val_predictions == False) & (labels == True)).sum().item()
        
        correct += (val_predictions == labels).sum().item() #count how many correct predictions are there
        total += labels.size(0)
        break

# Calculate precision, recall, and accuracy
print(tp,tn,fp,fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
print(correct,total)
accuracy = correct / total


print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

0 6 0 1
6 7
Accuracy: 0.8571
Precision: 0.0000
Recall: 0.0000


In [261]:
# Model gets overfitted to the class 0 since it is dominating in all the graphs
# need to select the developers who has diversed connections 